In [4]:
import pandas as pd

#  Todesursachen

In [1]:
with open("cube_23211.txt", "r", encoding="utf-8") as fh:
    raw = fh.read()
    
raw.splitlines()[:10]

['* Der Benutzer DEI6I4B3UW der Benutzergruppe DE0142 hat am 16.05.2022 um 08:24:13 diesen Export angestossen.',
 'K;DQ;FACH-SCHL;GHH-ART;GHM-WERTE-JN;GENESIS-VBD;REGIOSTAT;EU-VBD;"nur Werte"',
 'D;23211BJ001;;N;J;N;N',
 'K;DQ-ERH;FACH-SCHL',
 'D;23211',
 'K;DQA;NAME;RHF-BSR;RHF-ACHSE',
 'D;DINSG;1;1',
 'D;GES;2;2',
 'D;ALT027;3;3',
 'D;TODUR1;4;4']

In [53]:
# find column names (use unique variable codes)
dqa_names = []
is_metadata = False
for line in raw.splitlines():
    if is_metadata:
        if line.startswith("K"):
            break
        
        dqa_names.append(line.split(";")[1])
        
    if "DQA" in line.split(";"):
        is_metadata = True
        

In [54]:
dqa_names

['DINSG', 'GES', 'ALT027', 'TODUR1']

In [52]:
# find column names (use unique variable codes)
# time
dqz_names = []
is_metadata = False
for line in raw.splitlines():
    if is_metadata:
        if line.startswith("K"):
            break
        
        dqz_names.append(line.split(";")[1])
        
    if "DQZ" in line.split(";"):
        is_metadata = True
        

In [55]:
dqz_names

['JAHR']

In [24]:
# find row with data header
row_header = 0
for i, line in enumerate(raw.splitlines()):
    if "QEI" in line.split(";"):
        row_header = i
    

In [25]:
header = ";".join(raw.splitlines()[row_header].split(";")[2:])


In [39]:
from io import StringIO

with StringIO(raw) as stream:
    df = pd.read_csv(stream, skiprows=row_header, sep=";")
    df = df.drop(columns=["K"])

In [40]:
df

,QEI,FACH-SCHL,FACH-SCHL.1,FACH-SCHL.2,FACH-SCHL.3,ZI-WERT,WERT,QUALITAET,GESPERRT,WERT-VERFAELSCHT
0,DG,GESM,ALT000,TODESURS01,1980,6205,e,NaN,0,NaN
1,DG,GESM,ALT000,TODESURS01,1981,5915,e,NaN,0,NaN
2,DG,GESM,ALT000,TODESURS01,1982,5484,e,NaN,0,NaN
3,DG,GESM,ALT000,TODESURS01,1983,4998,e,NaN,0,NaN
4,DG,GESM,ALT000,TODESURS01,1984,4529,e,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...
96724,DG,GESW,ALTNN,TODESURS80,1980,2,e,NaN,0,NaN
96725,DG,GESW,ALTNN,TODESURS80,1981,1,e,NaN,0,NaN
96726,DG,GESW,ALTNN,TODESURS80,1982,1,e,NaN,0,NaN
96727,DG,GESW,ALTNN,TODESURS80,1983,1,e,NaN,0,NaN


In [41]:
df.iloc[:,1:] = df.iloc[:,:-1]
df = df.drop(columns=["QEI"])

In [42]:
df

,FACH-SCHL,FACH-SCHL.1,FACH-SCHL.2,FACH-SCHL.3,ZI-WERT,WERT,QUALITAET,GESPERRT,WERT-VERFAELSCHT
0,DG,GESM,ALT000,TODESURS01,1980,6205,e,NaN,0
1,DG,GESM,ALT000,TODESURS01,1981,5915,e,NaN,0
2,DG,GESM,ALT000,TODESURS01,1982,5484,e,NaN,0
3,DG,GESM,ALT000,TODESURS01,1983,4998,e,NaN,0
4,DG,GESM,ALT000,TODESURS01,1984,4529,e,NaN,0
...,...,...,...,...,...,...,...,...,...
96724,DG,GESW,ALTNN,TODESURS80,1980,2,e,NaN,0
96725,DG,GESW,ALTNN,TODESURS80,1981,1,e,NaN,0
96726,DG,GESW,ALTNN,TODESURS80,1982,1,e,NaN,0
96727,DG,GESW,ALTNN,TODESURS80,1983,1,e,NaN,0


In [59]:
col_old = [c for c in df.columns if c.startswith("FACH-SCHL")]

df = df.rename(columns={col_old: col_new for col_old, col_new in zip(col_old, dqa_names)})

In [60]:
df

,DINSG,GES,ALT027,TODUR1,JAHR,WERT,QUALITAET,GESPERRT,WERT-VERFAELSCHT
0,DG,GESM,ALT000,TODESURS01,1980,6205,e,NaN,0
1,DG,GESM,ALT000,TODESURS01,1981,5915,e,NaN,0
2,DG,GESM,ALT000,TODESURS01,1982,5484,e,NaN,0
3,DG,GESM,ALT000,TODESURS01,1983,4998,e,NaN,0
4,DG,GESM,ALT000,TODESURS01,1984,4529,e,NaN,0
...,...,...,...,...,...,...,...,...,...
96724,DG,GESW,ALTNN,TODESURS80,1980,2,e,NaN,0
96725,DG,GESW,ALTNN,TODESURS80,1981,1,e,NaN,0
96726,DG,GESW,ALTNN,TODESURS80,1982,1,e,NaN,0
96727,DG,GESW,ALTNN,TODESURS80,1983,1,e,NaN,0


In [57]:
col_old = [c for c in df.columns if c.startswith("ZI-WERT")]

df = df.rename(columns={col_old: col_new for col_old, col_new in zip(col_old, dqz_names)})

In [68]:
df

,DINSG,GES,ALT027,TODUR1,JAHR,WERT,QUALITAET,GESPERRT,WERT-VERFAELSCHT
0,DG,GESM,ALT000,TODESURS01,1980,6205,e,NaN,0
1,DG,GESM,ALT000,TODESURS01,1981,5915,e,NaN,0
2,DG,GESM,ALT000,TODESURS01,1982,5484,e,NaN,0
3,DG,GESM,ALT000,TODESURS01,1983,4998,e,NaN,0
4,DG,GESM,ALT000,TODESURS01,1984,4529,e,NaN,0
...,...,...,...,...,...,...,...,...,...
96724,DG,GESW,ALTNN,TODESURS80,1980,2,e,NaN,0
96725,DG,GESW,ALTNN,TODESURS80,1981,1,e,NaN,0
96726,DG,GESW,ALTNN,TODESURS80,1982,1,e,NaN,0
96727,DG,GESW,ALTNN,TODESURS80,1983,1,e,NaN,0


In [67]:
df["QUALITAET"].value_counts()

e    96483
.      246
Name: QUALITAET, dtype: int64

In [77]:
df.query("JAHR == 1980 and TODUR1 == 'TODESURS01'")

,DINSG,GES,ALT027,TODUR1,JAHR,WERT,QUALITAET,GESPERRT,WERT-VERFAELSCHT
0,DG,GESM,ALT000,TODESURS01,1980,6205,e,NaN,0
1761,DG,GESM,ALT001B15,TODESURS01,1980,3076,e,NaN,0
4044,DG,GESM,ALT015B20,TODESURS01,1980,3965,e,NaN,0
6426,DG,GESM,ALT020B25,TODESURS01,1980,4521,e,NaN,0
9117,DG,GESM,ALT025B30,TODESURS01,1980,3762,e,NaN,0
11923,DG,GESM,ALT030B35,TODESURS01,1980,3721,e,NaN,0
14815,DG,GESM,ALT035B40,TODESURS01,1980,6039,e,NaN,0
17769,DG,GESM,ALT040B45,TODESURS01,1980,10964,e,NaN,0
20775,DG,GESM,ALT045B50,TODESURS01,1980,14050,e,NaN,0
23789,DG,GESM,ALT050B55,TODESURS01,1980,20153,e,NaN,0


In [86]:
df[df.TODUR1 != "TODESURS01"].groupby(["JAHR", "TODUR1"]).WERT.sum().groupby("JAHR").nlargest(3)

JAHR  JAHR  TODUR1    
1980  1980  TODESURS40    500481
            TODESURS07    201084
            TODESURS08    192719
1981  1981  TODESURS40    505229
            TODESURS07    202255
                           ...  
2019  2019  TODESURS07    239591
            TODESURS08    231318
2020  2020  TODESURS40    338001
            TODESURS07    239552
            TODESURS08    231271
Name: WERT, Length: 123, dtype: int64

# Zensus

In [87]:
with open("cube_12111.txt", "r", encoding="utf-8") as fh:
    raw = fh.read()
    
raw.splitlines()[:10]

['* Der Benutzer DEI6I4B3UW der Benutzergruppe DE0142 hat am 16.05.2022 um 09:23:07 diesen Export angestossen.',
 'K;DQ;FACH-SCHL;GHH-ART;GHM-WERTE-JN;GENESIS-VBD;REGIOSTAT;EU-VBD;"nur Werte"',
 'D;12111B1012;;N;N;N;N',
 'K;DQ-ERH;FACH-SCHL',
 'D;12111',
 'K;DQA;NAME;RHF-BSR;RHF-ACHSE',
 'D;DINSG;1;1',
 'D;ALT041;2;2',
 'D;FAMST2;3;3',
 'K;DQZ;NAME;ZI-RHF-BSR;ZI-RHF-ACHSE']

In [90]:
# find column names (use unique variable codes)
dqa_names = []
is_metadata = False
for line in raw.splitlines():
    if is_metadata:
        if line.startswith("K"):
            break
        
        dqa_names.append(line.split(";")[1])
        
    if "DQA" in line.split(";"):
        is_metadata = True
        

In [91]:
dqa_names

['DINSG', 'ALT041', 'FAMST2']

In [92]:
# find column names (use unique variable codes)
# time
dqz_names = []
is_metadata = False
for line in raw.splitlines():
    if is_metadata:
        if line.startswith("K"):
            break
        
        dqz_names.append(line.split(";")[1])
        
    if "DQZ" in line.split(";"):
        is_metadata = True
        

In [93]:
dqz_names

['STAG']

In [94]:
# find row with data header
row_header = 0
for i, line in enumerate(raw.splitlines()):
    if "QEI" in line.split(";"):
        row_header = i
    

In [95]:
header = ";".join(raw.splitlines()[row_header].split(";")[2:])


In [96]:
from io import StringIO

with StringIO(raw) as stream:
    df = pd.read_csv(stream, skiprows=row_header, sep=";")
    df = df.drop(columns=["K"])

In [97]:
df

,QEI,FACH-SCHL,FACH-SCHL.1,FACH-SCHL.2,ZI-WERT,WERT,QUALITAET,GESPERRT,WERT-VERFAELSCHT
0,DG,ALT000B18,GESCH-L,09.05.2011,19,e,NaN,0,NaN
1,DG,ALT000B18,LEDIG,09.05.2011,13131811,e,NaN,0,NaN
2,DG,ALT000B18,OA,09.05.2011,2143,e,NaN,0,NaN
3,DG,ALT000B18,VERH-L,09.05.2011,367,e,NaN,0,NaN
4,DG,ALT000B18,VERW-L,09.05.2011,12,e,NaN,0,NaN
5,DG,ALT018B30,GESCH-L,09.05.2011,93942,e,NaN,0,NaN
6,DG,ALT018B30,LEDIG,09.05.2011,9945132,e,NaN,0,NaN
7,DG,ALT018B30,OA,09.05.2011,5472,e,NaN,0,NaN
8,DG,ALT018B30,VERH-L,09.05.2011,1345596,e,NaN,0,NaN
9,DG,ALT018B30,VERW-L,09.05.2011,2849,e,NaN,0,NaN


In [98]:
df.iloc[:,1:] = df.iloc[:,:-1]
df = df.drop(columns=["QEI"])

In [99]:
df

,FACH-SCHL,FACH-SCHL.1,FACH-SCHL.2,ZI-WERT,WERT,QUALITAET,GESPERRT,WERT-VERFAELSCHT
0,DG,ALT000B18,GESCH-L,09.05.2011,19,e,NaN,0
1,DG,ALT000B18,LEDIG,09.05.2011,13131811,e,NaN,0
2,DG,ALT000B18,OA,09.05.2011,2143,e,NaN,0
3,DG,ALT000B18,VERH-L,09.05.2011,367,e,NaN,0
4,DG,ALT000B18,VERW-L,09.05.2011,12,e,NaN,0
5,DG,ALT018B30,GESCH-L,09.05.2011,93942,e,NaN,0
6,DG,ALT018B30,LEDIG,09.05.2011,9945132,e,NaN,0
7,DG,ALT018B30,OA,09.05.2011,5472,e,NaN,0
8,DG,ALT018B30,VERH-L,09.05.2011,1345596,e,NaN,0
9,DG,ALT018B30,VERW-L,09.05.2011,2849,e,NaN,0


In [100]:
col_old = [c for c in df.columns if c.startswith("FACH-SCHL")]

df = df.rename(columns={col_old: col_new for col_old, col_new in zip(col_old, dqa_names)})

In [101]:
df

,DINSG,ALT041,FAMST2,ZI-WERT,WERT,QUALITAET,GESPERRT,WERT-VERFAELSCHT
0,DG,ALT000B18,GESCH-L,09.05.2011,19,e,NaN,0
1,DG,ALT000B18,LEDIG,09.05.2011,13131811,e,NaN,0
2,DG,ALT000B18,OA,09.05.2011,2143,e,NaN,0
3,DG,ALT000B18,VERH-L,09.05.2011,367,e,NaN,0
4,DG,ALT000B18,VERW-L,09.05.2011,12,e,NaN,0
5,DG,ALT018B30,GESCH-L,09.05.2011,93942,e,NaN,0
6,DG,ALT018B30,LEDIG,09.05.2011,9945132,e,NaN,0
7,DG,ALT018B30,OA,09.05.2011,5472,e,NaN,0
8,DG,ALT018B30,VERH-L,09.05.2011,1345596,e,NaN,0
9,DG,ALT018B30,VERW-L,09.05.2011,2849,e,NaN,0


In [102]:
col_old = [c for c in df.columns if c.startswith("ZI-WERT")]

df = df.rename(columns={col_old: col_new for col_old, col_new in zip(col_old, dqz_names)})

In [103]:
df

,DINSG,ALT041,FAMST2,STAG,WERT,QUALITAET,GESPERRT,WERT-VERFAELSCHT
0,DG,ALT000B18,GESCH-L,09.05.2011,19,e,NaN,0
1,DG,ALT000B18,LEDIG,09.05.2011,13131811,e,NaN,0
2,DG,ALT000B18,OA,09.05.2011,2143,e,NaN,0
3,DG,ALT000B18,VERH-L,09.05.2011,367,e,NaN,0
4,DG,ALT000B18,VERW-L,09.05.2011,12,e,NaN,0
5,DG,ALT018B30,GESCH-L,09.05.2011,93942,e,NaN,0
6,DG,ALT018B30,LEDIG,09.05.2011,9945132,e,NaN,0
7,DG,ALT018B30,OA,09.05.2011,5472,e,NaN,0
8,DG,ALT018B30,VERH-L,09.05.2011,1345596,e,NaN,0
9,DG,ALT018B30,VERW-L,09.05.2011,2849,e,NaN,0


In [104]:
df["ALT041"].value_counts()

ALT000B18    5
ALT018B30    5
ALT030B50    5
ALT050B65    5
ALT065UM     5
Name: ALT041, dtype: int64

In [112]:
df.groupby(["STAG", "ALT041", "FAMST2"]).WERT.sum()

STAG        ALT041     FAMST2 
09.05.2011  ALT000B18  GESCH-L          19
                       LEDIG      13131811
                       OA             2143
                       VERH-L          367
                       VERW-L           12
            ALT018B30  GESCH-L       93942
                       LEDIG       9945132
                       OA             5472
                       VERH-L      1345596
                       VERW-L         2849
            ALT030B50  GESCH-L     2275423
                       LEDIG       6644276
                       OA            11471
                       VERH-L     13760646
                       VERW-L       153856
            ALT050B65  GESCH-L     2189239
                       LEDIG       1508128
                       OA             4397
                       VERH-L     11872166
                       VERW-L       754629
            ALT065UM   GESCH-L     1124562
                       LEDIG        809744
                       